In [0]:
import os

In [0]:
ENV = "dev"
bronze_schema = f"fpl_bronze_{ENV}"

In [0]:
# Base path to your volume
base_path = "/Volumes/workspace/fpl_raw/player_data"
output_base = f"{bronze_schema}"

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)

    # Skip non-directories
    if not os.path.isdir(folder_path):
        continue

    # Extract season years
    try:
        start_year, end_year = folder.split("_")
        start_year_int = int(start_year)
    except ValueError:
        print(f"Skipping folder: {folder}")
        continue

    # Skip folders from 2025_26 onwards
    if start_year_int >= 2025:
        print(f"Skipping folder: {folder}")
        continue

    start_year_short = start_year[2:]
    csv_file = f"all_player_stats_{start_year_short}_{end_year}.csv"
    csv_path = os.path.join(folder_path, csv_file)

    try:
        df = spark.read.option("header", True).option("encoding", "utf-8").csv(csv_path) #set schema or all string?
    except Exception as e:
        print(f"Failed to read {csv_path}: {e}")
        continue

    delta_table_name = f"{output_base}.player_gameweek_stats_{start_year_short}_{end_year}"
    print("Writing to:", delta_table_name)

    df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
    print(f"Written Delta table: player_gameweek_stats_{start_year_short}_{end_year}")

    csv_file = f"players_raw_{start_year_short}_{end_year}.csv"
    csv_path = os.path.join(folder_path, csv_file)

    try:
        df = spark.read.option("header", True).option("encoding", "utf-8").csv(csv_path) #set schema or all string?
    except Exception as e:
        print(f"Failed to read {csv_path}: {e}")
        continue

    delta_table_name = f"{output_base}.players_raw_{start_year_short}_{end_year}"
    print("Writing to:", delta_table_name)

    df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
    print(f"Written Delta table: players_raw_{start_year_short}_{end_year}")

Writing to: fpl_bronze_dev.player_gameweek_stats_16_17
Written Delta table: player_gameweek_stats_16_17
Writing to: fpl_bronze_dev.players_raw_16_17
Written Delta table: players_raw_16_17
Writing to: fpl_bronze_dev.player_gameweek_stats_17_18
Written Delta table: player_gameweek_stats_17_18
Writing to: fpl_bronze_dev.players_raw_17_18
Written Delta table: players_raw_17_18
Writing to: fpl_bronze_dev.player_gameweek_stats_18_19
Written Delta table: player_gameweek_stats_18_19
Writing to: fpl_bronze_dev.players_raw_18_19
Written Delta table: players_raw_18_19
Writing to: fpl_bronze_dev.player_gameweek_stats_19_20
Written Delta table: player_gameweek_stats_19_20
Writing to: fpl_bronze_dev.players_raw_19_20
Written Delta table: players_raw_19_20
Writing to: fpl_bronze_dev.player_gameweek_stats_20_21
Written Delta table: player_gameweek_stats_20_21
Writing to: fpl_bronze_dev.players_raw_20_21
Written Delta table: players_raw_20_21
Writing to: fpl_bronze_dev.player_gameweek_stats_21_22
Writt